In [21]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings
import nest_asyncio
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [22]:
nest_asyncio.apply()

In [23]:
# ollama
Settings.llm = Ollama(model="qwen2:0.5b", request_timeout=360.0)

# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [24]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [25]:
# load data
pg_essay = SimpleDirectoryReader(input_dir="./data").load_data()

# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    pg_essay,
    use_async=True,
).as_query_engine()

**********
Trace: index_construction
    |_embedding -> 13.896819 seconds
    |_embedding -> 13.897055 seconds
    |_embedding -> 13.897106 seconds
**********


In [26]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="pg_essay",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [32]:
response = query_engine.query(
    "How was Paul Grahams life different before, during, and after YC?"
)

Generated 3 sub questions.
[pg_essay] Q: Before YC
[pg_essay] Q: During YC
[pg_essay] Q: After YC
[pg_essay] A: After YC.
[pg_essay] A: Before YC
[pg_essay] A: Based on the context provided in the document, during YC (the startup consulting firm) there were two main projects that were working at the same time:

1. "YC was different from other kinds of work I've done." This suggests that YC was focusing on a very specific type of consulting and development work.

2. There was also a "new version of Arc" that was being developed by Robert Morris. This indicates that Robert is actively working on new or revised versions of the software they are working on.

So in summary, during YC there were two main projects: a consulting firm focused on developing new software, and a development team developing an update to the existing software.
**********
Trace: query
    |_query -> 136.049625 seconds
      |_templating -> 3e-05 seconds
      |_llm -> 2.424401 seconds
      |_sub_question -> 104.0169

In [33]:
print(response)

Before YC:

Paul Graham is a well-known American computer scientist who founded the startup consulting firm called Y Combinator. He was born on December 26, 1970.

During his time at Y C, Paul worked with several notable figures in the tech industry such as Tim Berners-Lee, Elon Musk, Larry Page and Sergey Brin.

After YC:

Paul's tenure at Y C lasted from 2004 to 2008. He was responsible for shaping the direction of the firm. 

During this time he worked with several more notable figures in the tech industry like Steve Jobs, Jeff Bezos, and Larry Page. In addition, he helped bring about the creation of a new venture called "Blue Brain" which would later become known as Google.

Paul's work at Y C had a significant impact on the tech industry and played a crucial role in its development. His leadership style and emphasis on innovation were instrumental in the firm's success.

After YC:

In his time at Y C, Paul continued to be involved with many more prominent figures in the tech indus

In [34]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.core.callbacks import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Sub Question 0: Before YC
Answer: Before YC
Sub Question 1: During YC
Answer: Based on the context provided in the document, during YC (the startup consulting firm) there were two main projects that were working at the same time:

1. "YC was different from other kinds of work I've done." This suggests that YC was focusing on a very specific type of consulting and development work.

2. There was also a "new version of Arc" that was being developed by Robert Morris. This indicates that Robert is actively working on new or revised versions of the software they are working on.

So in summary, during YC there were two main projects: a consulting firm focused on developing new software, and a development team developing an update to the existing software.
Sub Question 2: After YC
Answer: After YC.
